Connected to Python

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [10]:
df=pd.read_csv('kc_house_train_data.csv')

Funcion para obtener una matriz de observaciones y el vector de resultados


In [11]:
def get_numpy_data(data_sframe,features,output):
    data_sframe['constant']=1 #agregamos un columna de 1 , este coeficientemultiplica al intercepto
    features=['constant']+features
    features_matrix=data_sframe[features].to_numpy()
    output_array=data_sframe[output].to_numpy()
    return(features_matrix,output_array)

Funcion para obtener las predicciones

In [12]:
def predict_outcome(feature_matrix,weights):
    predictions=np.dot(feature_matrix,weights)
    return predictions

In [13]:
#ejemplo de uso de las funciones anteriores (no usa los datos del dataframe)
try1=get_numpy_data(df,['bedrooms','bathrooms'],['price'])[0]
predict_outcome(try1,np.array([1,2,3]))


array([10.  , 13.75,  8.  , ..., 14.5 , 14.5 ,  7.25])

Funcion para obtener la derivada de una caracteristica

In [6]:
def feature_derivative(feature,errors):
    derivative=2*np.dot(np.array(feature),np.array(errors))
    return derivative

Funcion de gradiente descendiente

In [14]:
def regression_gradient_descent(feature_matrix,output,initial_weights,step_size,tolerance):
    converged=False
    weights=np.array(initial_weights)
    while converged==False:
        errors=output-np.dot(feature_matrix,weights)
        gradient=-2*(np.dot(feature_matrix.T,errors)) #chekar la observación
        weights=weights-step_size*gradient
        gradient_magnitude=np.linalg.norm(gradient)
        if gradient_magnitude<tolerance:
            converged=True
    return weights

# Observación

matematicamente se concluyo que la gradiente para una matriz de caracteristicas es : -2 * Matriz de caract.transpuesta * errores . Entiendase por errores como a la resta en etre el valor real y la predicción con determinados pesos 

# Enfoque RIDGE con la gradiente descendiente

In [15]:
def rgr_gd_ridge(feature_matrix,output,initial_weights,alpha,step_size,tolerance):
    converged=False
    weights=np.array(initial_weights)
    while converged==False:
        errors=output-np.dot(feature_matrix,weights)
        gradient=-2*(np.dot(feature_matrix.T,errors)) 
        weights=weights*(1-2*step_size*alpha)-step_size*gradient
        gradient_magnitude=np.linalg.norm(gradient)
        if gradient_magnitude<tolerance:
            converged=True
    return weights

Pruebas

In [16]:
#estructuramos los datos (atencion en como deben ser ingresados los parametros de estas funciones)
matriz=get_numpy_data(df,['sqft_living','sqft_living15'],'price')[0]
salida=get_numpy_data(df,['sqft_living','sqft_living15'],'price')[1]

In [17]:
#una pequeña prueba de como es que salen los datos
get_numpy_data(df,['sqft_living','sqft_living15'],'price')[1]

array([221900., 538000., 180000., ..., 360000., 400000., 325000.])

Probamos para el enfoque de la gradiente descendiente unicamente, la gradiente descendiente RIDGE y usando la librería sklearn para comparar

In [18]:
regression_gradient_descent(matriz,salida,[-100000, 1, 1],4e-12,1e9)

array([-9.99999688e+04,  2.45072603e+02,  6.52795267e+01])

In [19]:
rgr_gd_ridge(matriz,salida,[-100000, 1, 1],1.23e2,4e-12,1e9)

array([-9.99999419e+04,  2.45072599e+02,  6.52795195e+01])

In [22]:
rgr_lib=LinearRegression()
rgr_lib.fit(df[['sqft_living','sqft_living15']],salida)
rgr_lib.coef_

array([245.18871442,  65.27158522])

In [23]:
rgr_lib.intercept_

-100262.17515853501

Notamos que por en todos los  enfoques se obtiene los pesos bastantes similares , siendo que el enfoque ridge mejora la gradiente descendiente 